In [1]:
import pandas as pd
import geopandas as gp
import numpy as np
import matplotlib.pyplot as plt
import maup

# Mississippi 2015 Commissioner of Agriculture and Commerce Precinct Boundary and Election Results

In [2]:
BLOCK_PATH = "/Users/peterhorton/Documents/RDH/raw_data/census/2020_TIGER_B/ms_b_2020_bound.zip"

In [3]:
# Load census blocks
ms_blocks = gp.read_file(BLOCK_PATH)

# Load VEST's 2016 Mississippi shapefile
ms_vest = gp.read_file("./raw-from-source/ms_vest_16/ms_vest_16.shp")

# Load cleaned Mississippi election results via OpenElections
ms_results = pd.read_csv("./raw-from-source/20151103__ms__general__precinct_CLEAN.csv")

## Test which counties don't have particular statewide races

In [4]:
wanted = set(['Attorney General','Commissioner of Agriculture & Commerce',
 'Commissioner of Insurance','Governor','Lieutenant Governor',
 'Secretary of State','State Auditor','State Treasurer'])
for county in list(ms_results["county"].unique()):
    if len(wanted.difference(set(ms_results[ms_results["county"]==county]["office"].unique()))) != 0:
        print(county)
        print(wanted.difference(set(ms_results[ms_results["county"]==county]["office"].unique())))

Amite
{'Commissioner of Insurance'}
Attala
{'Commissioner of Insurance'}
Pearl River
{'Commissioner of Insurance'}
Simpson
{'Commissioner of Insurance'}


## Process Election Results

In [5]:
## Filter out non-statewide results
ms_results = ms_results[~ms_results["office"].isin(['State House','State Senate'])]

In [6]:
# Clean up contest typos
ms_results.loc[(ms_results["candidate"]=="Jim Hood")&(ms_results["office"]=="Secretary of State"), "office"] = "Attorney General"
ms_results.loc[(ms_results["candidate"]=="Mike Hurst")&(ms_results["office"]=="Secretary of State"),"office"] = "Attorney General"

# Clean up issue with district field overlapping with precinct name
ms_results["precinct"] = np.where(~ms_results["district"].isna(), ms_results["precinct"]+ " " +ms_results["district"], ms_results["precinct"])

# Define a UNIQUE_ID column
ms_results["UNIQUE_ID"] = ms_results["county"] + "-:-" + ms_results["precinct"]

# Pivot the dataframe and reset the index
ms_results_pivoted = pd.pivot_table(ms_results, index = ["UNIQUE_ID"])
ms_results_pivoted.reset_index(inplace = True)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_12664/111707237.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ms_results_pivoted = pd.pivot_table(ms_results, index = ["UNIQUE_ID"])


## Filter down to just Commissioner of Agriculture and Commerce Race

This is the only race where the results have been fully cleaned of typos

In [7]:
ms_results = ms_results[ms_results["office"].isin(['Commissioner of Agriculture & Commerce'])]

## Add in County Names to VEST File

In [8]:
# Load in csv with the county names
county_names = pd.read_csv("./raw-from-source/PL_COUNTYFP_NAMES.csv")

# Filter down to MS, zfill FIP, define a dictionary and use the dictionary
ms = county_names[county_names["STUSAB"]=="MS"]
ms["COUNTYFP"] = ms["COUNTYFP"].astype(str).str.zfill(3)
ms_dict = dict(zip(ms["COUNTYFP"], ms["NAME"].astype(str).str.replace(" County","")))
ms_vest["County_Name"] = ms_vest["COUNTYFP16"].map(ms_dict).fillna("NONE")

# Define a UNIQUE_ID column for the VEST file, used to join to election results
ms_vest["UNIQUE_ID"] = ms_vest["County_Name"] + "-:-"+ms_vest["NAME16"]

# For earlier work, export to csv to create a correspondence across the UNIQUE_ID fields
# ms_vest.to_csv("./shapefile_2016.csv", index = False)
# ms_results.to_csv("./results_2015.csv", index = False)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_12664/805346303.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ms["COUNTYFP"] = ms["COUNTYFP"].astype(str).str.zfill(3)


## Clean Election Results

In [9]:
# Remove a precinct that contains the total votes for a county
ms_results = ms_results[ms_results["UNIQUE_ID"]!="Pearl River-:-TOTAL"]

## Fix Typos and Merge Precincts

Notes from VEST processing
- Attala: Merge Northwest/Aponaug
- Hinds: Merge 12/13, 16/17, 26/87, 50/51/52, 67/89, 69/75, 94/95, 77/97
- Jefferson Davis: In Jefferson Davis County Bassfield 31/32 was an alphabetic name split of Bassfield Hwy 42.

In [10]:
typo_fix_dict = {"Forrest-:-Kemper Park":"Forrest-:-Kamper Park",
"Forrest-:-Leesville":"Forrest-:-Leeville",
"Forrest-:-Little Burney":"Forrest-:-Lillie Burney",
"Forrest-:-Siglar Center":"Forrest-:-Sigler Center",
"Lauderdale-:-262 - Toomsubba":"Lauderdale-:-262 - Toomsuba",
"Lawrence-:-Monticello-Beat 5":"Lawrence-:-Monticello - Beat 5",
"Pearl River-:-Anchor Lake/West Union Beat 5":"Pearl River-:-Anchor Lake / West Union Beat 5",
"Pearl River-:-Carriers Beat 5":"Pearl River-:-Carriere Beat 5",
"Pearl River-:-Savanna Beat 3":"Pearl River-:-Savannah Beat 3",
"Pearl River-:-Sleep Hollow Beat 3":"Pearl River-:-Steep Hollow Beat 3",
"Pearl River-:-Berby Bt. 1":"Pearl River-:-Derby Bt 1",
"Pearl River-:-Gumpound Beat 3":"Pearl River-:-Gumpond Beat 3",
"Madison-:-New Life 105":"Madison-:-New Life",           
# Merges
"Jefferson Davis-:-Bassfield 31":"Jefferson Davis-:-Bassfield 32",
"Hinds-:-Jackson prec 95":"Hinds-:-Jackson prec 94",
"Hinds-:-Jackson prec 97":"Hinds-:-Jackson prec 77",
"Attala-:-Aponaug":"Attala-:-North West"}

In [11]:
ms_results.loc[:,"UNIQUE_ID"] = ms_results["UNIQUE_ID"].map(typo_fix_dict).fillna(ms_results["UNIQUE_ID"])

## Pivot Election Results

In [12]:
# Clean typo in the party names
ms_results.loc[ms_results["party"]=="Cin","party"] = "Republican"

In [13]:
# Standardize the party names
party_update_dict = {'REF':'Reform',
 'LIB':'Libertarian',
 'libertarian':'Libertarian'}

ms_results["party"] = ms_results["party"].map(party_update_dict).fillna(ms_results["party"])

# Clean additional party typos
ms_results.loc[ms_results["candidate"]=='Mark Baker',"party"] = "Republican"
ms_results.loc[ms_results["candidate"]=='Dean Kirby', "party"] = "Republican"
ms_results.loc[ms_results["candidate"]=='Josh Harkins',"party"] = "Republican"

In [14]:
# Clean candidate names
cand_update_dict = {'w':'Write-in'}
ms_results["candidate"] = ms_results["candidate"].map(cand_update_dict).fillna(ms_results["candidate"])

In [15]:
# Standardize the use of write-in columns
ms_results.loc[(ms_results["party"].isna()) & (ms_results["candidate"].isin(['Write-in'])),"party"] = "WRI"

# If State House or State Senate eventually included, this includes district in the party name
ms_results["office_full"] = np.where(ms_results["office"].isin(['State House',
       'State Senate']), ms_results["office"]+" District "+ms_results["district"],ms_results["office"])

# Define a candidate column with the full office name, candidate and party
ms_results["cand_col"] = ms_results["office_full"] + "-:-" + ms_results["candidate"] + "-:-" + ms_results["party"]

In [16]:
# Fix various issues with the candidate names
cand_col_updates =  {'Commissioner of Agriculture & Commerce-:-Addie Voting Precinct Lee Green-:-Democrat':'Commissioner of Agriculture & Commerce-:-Addie Lee Green-:-Democrat',
 'Commissioner of Agriculture & Commerce-:-Addle Lee Green-:-Democrat':'Commissioner of Agriculture & Commerce-:-Addie Lee Green-:-Democrat',
 'Commissioner of Agriculture & Commerce-:-Cathy L Toole-:-Reform':'Commissioner of Agriculture & Commerce-:-Cathy L. Toole-:-Reform',
 'Commissioner of Agriculture & Commerce-:-Cathy Toole-:-Reform':'Commissioner of Agriculture & Commerce-:-Cathy L. Toole-:-Reform',
 'Commissioner of Agriculture & Commerce-:-Andy Hyde-Smith-:-Republican':'Commissioner of Agriculture & Commerce-:-Cindy Hyde-Smith-:-Republican',
 'Commissioner of Agriculture & Commerce-:-Cindy Hyd-Smith-:-Republican':'Commissioner of Agriculture & Commerce-:-Cindy Hyde-Smith-:-Republican',
 'Commissioner of Agriculture & Commerce-:-Cindy Hyde-Simth-:-Republican':'Commissioner of Agriculture & Commerce-:-Cindy Hyde-Smith-:-Republican',
 'Governor-:-Paul Bryant-:-Republican':'Governor-:-Phil Bryant-:-Republican',
 'Governor-:-Phil Bryant-:-Democrat':'Governor-:-Phil Bryant-:-Republican',
 "Governor-:-Shawn O'Harra-:-Reform":"Governor-:-Shawn O'Hara-:-Reform", 
 'Lieutenant Governor-:-Rosa B Williams-:-Reform':'Lieutenant Governor-:-Rosa B. Williams-:-Reform',
 'Lieutenant Governor-:-Rosa B. Walker-:-Reform':'Lieutenant Governor-:-Rosa B. Williams-:-Reform',
 'Lieutenant Governor-:-Rosa B.Williams-:-Reform':'Lieutenant Governor-:-Rosa B. Williams-:-Reform',
 'Lieutenant Governor-:-Rosa S. Williams-:-Reform':'Lieutenant Governor-:-Rosa B. Williams-:-Reform',
 'Lieutenant Governor-:-Tabe Reeves-:-Republican':'Lieutenant Governor-:-Tate Reeves-:-Republican',
 'Secretary of State-:-Charles E. Grahm-:-Democrat':'Secretary of State-:-Charles E. Graham-:-Democrat',
 'Secretary of State-:-Charles Graham-:-Democrat':'Secretary of State-:-Charles E. Graham-:-Democrat',
 'Secretary of State-:-Delbert Hoseman-:-Republican':'Secretary of State-:-Delbert Hosemann-:-Republican',
 'Secretary of State-:-Randy Walter-:-Reform':'Secretary of State-:-Randy Walker-:-Reform',
 'Secretary of State-:-Rnady Walker-:-Reform':'Secretary of State-:-Randy Walker-:-Reform',                   
 'State Auditor-:-J Pepper Pritchett-:-Democrat':'State Auditor-:-Jocelyn Pepper Pritchett-:-Democrat',
 'State Auditor-:-Jocelyn "Joce" Pepper Pritchett-:-Democrat':'State Auditor-:-Jocelyn Pepper Pritchett-:-Democrat',
 "State Auditor-:-Jocelyn 'Joce' Pepper Pritchett-:-Democrat":'State Auditor-:-Jocelyn Pepper Pritchett-:-Democrat',
 "State Auditor-:-Joceyln 'Joce' Pepper Pritchett-:-Republican":'State Auditor-:-Jocelyn Pepper Pritchett-:-Democrat',
 'State Auditor-:-Joceyln Pepper Pritchett-:-Democrat':'State Auditor-:-Jocelyn Pepper Pritchett-:-Democrat',
 "State Auditor-:-Jocelyn 'Joyce' Pepper Pritchett-:-Democrat":'State Auditor-:-Jocelyn Pepper Pritchett-:-Democrat',
 'State Auditor-:-Joceyln Pepper Pritchett-:-Republican':'State Auditor-:-Jocelyn Pepper Pritchett-:-Democrat',
 'State Auditor-:-Stacey E. Pickering-:-Democrat':'State Auditor-:-Stacey Pickering-:-Republican',
 'State Auditor-:-Stacey E. Pickering-:-Republican':'State Auditor-:-Stacey Pickering-:-Republican',
 'State Auditor-:-Lajean Walley-:-Reform':'State Auditor-:-Lajena Walley-:-Reform',
 'State Auditor-:-Lajena Wallay-:-Reform':'State Auditor-:-Lajena Walley-:-Reform',
 'State Auditor-:-Lajena Walley-:-Republican':'State Auditor-:-Lajena Walley-:-Reform',                      
 'State Treasurer-:-Lynn Finch-:-Republican':'State Treasurer-:-Lynn Fitch-:-Republican',
 'State Treasurer-:-Viola McFarland-:-Reform':'State Treasurer-:-Viola V. McFarland-:-Reform',
 'State Treasurer-:-Viola V McFarland-:-Democrat':'State Treasurer-:-Viola V. McFarland-:-Reform',
 'State Treasurer-:-Viola V. McFarland-:-Democrat':'State Treasurer-:-Viola V. McFarland-:-Reform'}

# Carry out the above changes
ms_results["cand_col"] = ms_results["cand_col"].map(cand_col_updates).fillna(ms_results["cand_col"])

In [17]:
# Confirm that there are no remaining duplicate candidates
ms_results["cand_col"].unique()

array(['Commissioner of Agriculture & Commerce-:-Cathy L. Toole-:-Reform',
       'Commissioner of Agriculture & Commerce-:-Addie Lee Green-:-Democrat',
       'Commissioner of Agriculture & Commerce-:-Cindy Hyde-Smith-:-Republican',
       'Commissioner of Agriculture & Commerce-:-Write-in-:-WRI'],
      dtype=object)

In [18]:
# Pivot the election results
ms_results_pivoted = pd.pivot_table(ms_results, index = ["UNIQUE_ID","county"], columns = "cand_col", values = "votes", aggfunc = sum)

# Fillna with zero and reset index
ms_results_pivoted = ms_results_pivoted.fillna(0)
ms_results_pivoted.reset_index(inplace = True, drop = False)

## Rename columns to fit character limit

In [19]:
def get_race(race_string):
    race_string = race_string.title()
    race_string = race_string.replace("(Vote For 1)","")
    if "U.S. House" in race_string or 'Us House' in race_string:
        return "CON"
    elif "State House" in race_string:
        return "SL"
    elif "State Senate" in race_string:
        return "SU"
    elif "President" in race_string:
        return "PRE"
    elif "US Senate" in race_string or "Us Senate" in race_string:
        return "USS"
    elif "Public Service" in race_string:
        return "PSC"
    elif "Attorney General" in race_string:
        return "ATG"
    elif "Auditor General" in race_string or "Auditor" in race_string:
        return "AUD"
    elif "Treasurer" in race_string:
        return "TRE"
    elif "Superintendent" in race_string:
        return "SUP"
    elif "Secretary Of State" in race_string:
        return "SOS"
    elif "Lieutenant Governor" in race_string:
        return "LTG"
    elif "Governor" in race_string:
        return "GOV"
    elif "Commissioner Of Labor" in race_string:
        return "LAB"
    elif "Commissioner Of Agriculture" in race_string:
        return "AGR"
    elif "Commissioner Of Insurance" in race_string:
        return "INS"
    elif "Commissioner Of Insurance" in race_string:
        return "INS"
    elif "Amendment" in race_string:
        if "1" in race_string:
            return "A01"
        elif "2" in race_string:
            return "A02"
        else:
            print("No race for:", race_string)
            raise ValueError
    elif "Referendum" in race_string:
        if "1" in race_string or "A" in race_string:
            return "RFA"
        elif "2" in race_string or "B" in race_string:
            return "RFB"
        else:
            print("No race for:", race_string)
            raise ValueError
    else:
        print("No race for:", race_string)
        raise ValueError
        
def get_election_type(race_string):
    if "(runoff)" in race_string:
        return "R"
    else:
        return "G"
        
def get_party(race_string):
    race_string = race_string.split("-:-")[2]
    if race_string == "Reform":
        return "F"
    elif race_string == "WRI":
        return "O"
    else:
        return race_string[0]
           
def get_name(name_string):

    name_string = name_string.split("-:-")[1]
    name_string = name_string.replace("'","")
    likely_last = name_string.split(" ")[-1]
    proposed_last = likely_last[:3]
    if proposed_last in ['II', 'III', 'Jr', 'Jr.', 'Sr.', 'JR.', "JR", "IV"]:
        likely_last = name_string.split(" ")[-2]
        proposed_last = likely_last[:3]
    #print(proposed_last.upper())
    return proposed_last.upper()

#     name_string = name_string.split("-:-")[1]
#     name_string = name_string.replace(" (I)","")
#     name_string = name_string.replace("'","")
#     likely_last = name_string.split(" ")[-1]
#     proposed_last = likely_last[:3]
#     if proposed_last in ['II', 'III', 'Jr', 'Jr.', 'Sr.', 'JR.', "JR", "IV"]:
#         likely_last = name_string.split(" ")[-2]
#         proposed_last = likely_last[:3]
#     return proposed_last.upper()

def get_district(race_string, fill_level):
    race_string = race_string.replace(" (Vote For 1)","")
    if "Dist " in race_string:
        break_word = "Dist "
    elif "District " in race_string:
        break_word = "District "
    temp = race_string.split(break_word)[1]
    return temp.zfill(fill_level)

def column_rename_function(name_string):
    election_type = get_election_type(name_string)
    year = "15"
    party = get_party(name_string)
    race = get_race(name_string)
    district = ""
    if race in ["CON", "SU"]:
        district = get_district(name_string, 2)
        year = ""
    elif race in ["SL"]:
        district = get_district(name_string, 3)
        year = ""
    name = get_name(name_string)
    new_col_name = election_type + year + race + district + party + name
    if len(new_col_name) > 10:
        print(name_string)
        print(new_col_name)
    return new_col_name

# Make a dictionary that points to the new column names and checks for duplicates
race_columns = [i for i in ms_results_pivoted.columns if i not in ['UNIQUE_ID', 'county', 'COUNTYFP', 'precinct','Tot_Votes_15']]

race_updates_dict = {}
race_updates_reversed = {}
clean_dups = {}
new_names = []
for val in race_columns:
    new_name = column_rename_function(val)
    race_updates_dict[val] = new_name
    if new_name not in new_names:
        new_names.append(new_name)
        race_updates_reversed[new_name] = val
    else:
        print("Duplicate", new_name)
        print(race_updates_reversed[new_name])
        print(val)
        clean_dups[val] = race_updates_reversed[new_name]

In [20]:
# Confirm the renaming
race_updates_dict

{'Commissioner of Agriculture & Commerce-:-Addie Lee Green-:-Democrat': 'G15AGRDGRE',
 'Commissioner of Agriculture & Commerce-:-Cathy L. Toole-:-Reform': 'G15AGRFTOO',
 'Commissioner of Agriculture & Commerce-:-Cindy Hyde-Smith-:-Republican': 'G15AGRRHYD',
 'Commissioner of Agriculture & Commerce-:-Write-in-:-WRI': 'G15AGROWRI'}

In [21]:
# Rename the columns
ms_results_pivoted.rename(columns = race_updates_dict, inplace = True)

In [22]:
for col in list(race_updates_dict.values()):
    print(col + " " + str(int(sum(ms_results_pivoted[col]))))

G15AGRDGRE 260459
G15AGRFTOO 14969
G15AGRRHYD 436415
G15AGROWRI 71


Compare the totals in the file against the official ones, official after the dash, note only "AGR" races fully cleaned
- G15ATGDHOO 392920 - 400,110
- G15ATGRHUR 315672 - 322,648
- G15AGRDGRE 260459 - 260,584
- G15AGRFTOO 14969 - 14,975
- G15AGRRHYD 436415  -436,527
- G15INSRCHA 556684 - 591,566
- G15GOVRBRY 477720 - 480,399
- G15GOVDGRA 231907 - 234,858
- G15GOVFOHA 9844 - 9,950
- G15LTGLWIL 16264 - 16,317
- G15LTGFWIL 9355 - 9,480
- G15LTGRREE 423598 - 433,382
- G15LTGDJOH 253998 - 259,008
- G15SOSDGRA 251491 - 256,689
- G15SOSRHOS 429787 - 440,048
- G15SOSFWAL 20905 - 21,260
- G15AUDDPRI 248493 - 248,493
- G15AUDFWAL 9704 - 9,385
- G15AUDRPIC 443633 - 456,909
- G15TRERFIT 497213 - 516,666
- G15TREFMCF 144340 - 135,878

## Join File

Note: This match dictionary was constructed manually based on the precinct names, or in a handful of instances additional precinct information

In [23]:
join_data = pd.read_csv("./raw-from-source/ms_match_dictionary.csv")
join_dict = dict(zip(join_data["ELEC_ID"], join_data["VEST_ID"]))
ms_results_pivoted["UNIQUE_ID"] = ms_results_pivoted["UNIQUE_ID"].map(join_dict).fillna(ms_results_pivoted["UNIQUE_ID"])

#Holmes-:-2 Lexington Sandhill - contains 0 voters, fine to remove from results
ms_results_pivoted = ms_results_pivoted[ms_results_pivoted["UNIQUE_ID"]!="Holmes-:-2 Lexington Sandhill"]

# Filter down 
ms_vest = gp.GeoDataFrame(ms_vest[["UNIQUE_ID", "geometry"]])

# Groupby UNIQUE_ID
ms_results_pivoted = ms_results_pivoted.groupby("UNIQUE_ID").sum()
ms_results_pivoted.reset_index(inplace = True, drop = False)

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_12664/44544875.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ms_results_pivoted = ms_results_pivoted.groupby("UNIQUE_ID").sum()


In [24]:
join_one = gp.GeoDataFrame(pd.merge(ms_results_pivoted, ms_vest, how = "outer", on = "UNIQUE_ID", indicator = True))

In [25]:
join_one["_merge"].value_counts()

join_one.drop("_merge", axis = 1, inplace = True)

## Export File

In [26]:
join_one

,UNIQUE_ID,G15AGRDGRE,G15AGRFTOO,G15AGRRHYD,G15AGROWRI,geometry
0,Adams-:-Airport,224.0,1.0,108.0,0.0,"POLYGON ((-91.34813 31.63064, -91.34803 31.630..."
1,Adams-:-Beau Pre,128.0,6.0,263.0,0.0,"POLYGON ((-91.43870 31.44414, -91.43856 31.445..."
2,Adams-:-Bellemont,196.0,11.0,737.0,0.0,"POLYGON ((-91.42315 31.50152, -91.42313 31.501..."
3,Adams-:-By-Pass Firestation,242.0,4.0,128.0,0.0,"POLYGON ((-91.65501 31.25181, -91.65468 31.253..."
4,Adams-:-Carpenter,139.0,2.0,101.0,0.0,"POLYGON ((-91.40301 31.56649, -91.40293 31.566..."
...,...,...,...,...,...,...
1792,Yazoo-:-Ward 4 Teen Center,448.0,8.0,315.0,0.0,"POLYGON ((-90.40454 32.86526, -90.40484 32.865..."
1793,Yazoo-:-Ward 5 N D Taylor Gym,578.0,15.0,54.0,0.0,"POLYGON ((-90.41459 32.86611, -90.41388 32.866..."
1794,Yazoo-:-West Bentonia,166.0,12.0,179.0,0.0,"POLYGON ((-90.44207 32.59605, -90.44211 32.596..."
1795,Yazoo-:-West Midway,25.0,1.0,112.0,0.0,"POLYGON ((-90.25650 32.86860, -90.25636 32.868..."


In [27]:
join_one["County"] = join_one["UNIQUE_ID"].apply(lambda x: x.split("-:-")[0])
join_one["Precinct"] = join_one["UNIQUE_ID"].apply(lambda x: x.split("-:-")[1])

In [28]:
ms_dict_rev = {j:i for i,j in ms_dict.items()}

In [29]:
join_one["COUNTYFP"] = join_one["County"].map(ms_dict_rev).fillna("N/A")

In [30]:
join_one.reset_index(inplace = True, drop = True)

In [31]:
join_one.columns

Index(['UNIQUE_ID', 'G15AGRDGRE', 'G15AGRFTOO', 'G15AGRRHYD', 'G15AGROWRI',
       'geometry', 'County', 'Precinct', 'COUNTYFP'],
      dtype='object')

In [32]:
join_one = gp.GeoDataFrame(join_one[["UNIQUE_ID", "COUNTYFP", "County", 'Precinct','G15AGRDGRE', 'G15AGRFTOO', 'G15AGRRHYD', 'G15AGROWRI', "geometry"]])

In [ ]:
if not os.path.exists("./ms_2015_gen_prec"):
    os.mkdir("./ms_2015_gen_prec/")

In [33]:
# Export file
join_one.to_file("./ms_2015_gen_prec/ms_2015_gen_prec.shp")

/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/geopandas/io/file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [34]:
join_one = join_one.to_crs(ms_blocks.crs)
holder = maup.assign(ms_blocks, join_one)

/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:15: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  geometry.index = i
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:16: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  self.spatial_index = STRtree(self.geometries)
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:16: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  self.spatial_index = STRtree(self.geometries)
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it on

/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use 's.is_empty | s.isna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
  return intersections[-(intersections.is_empty | intersections.isna())]
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed com

/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use 's.is_empty | s.isna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
  return intersections[-(intersections.is_empty | intersections.isna())]
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed com

/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use 's.is_empty | s.isna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
  return intersections[-(intersections.is_empty | intersections.isna())]
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed com

/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use 's.is_empty | s.isna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
  return intersections[-(intersections.is_empty | intersections.isna())]
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed com

...

/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use 's.is_empty | s.isna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
  return intersections[-(intersections.is_empty | intersections.isna())]
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed com

/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use 's.is_empty | s.isna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
  return intersections[-(intersections.is_empty | intersections.isna())]
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed com

/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use 's.is_empty | s.isna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
  return intersections[-(intersections.is_empty | intersections.isna())]
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed com

/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use 's.is_empty | s.isna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
  return intersections[-(intersections.is_empty | intersections.isna())]
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/indexed_geometries.py:27: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed com

/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.8/site-packages/maup/assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)
